In [14]:
import pandas as pd
data = pd.read_csv("abstracts.csv")
data = data.drop('Unnamed: 0', axis=1)
data

,S,P,O,label,dboSource,Abstracts
0,http://yago-knowledge.org/resource/Van_Helsing...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,Van Helsing (film),Van_Helsing_(film),Van Helsing is a 2004 action gothic horror fil...
1,http://yago-knowledge.org/resource/One_Flew_Ov...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,One Flew Over the Cuckoo's Nest (film),One_Flew_Over_the_Cuckoo's_Nest_(film),One Flew Over the Cuckoo's Nest is a 1975 Amer...
2,http://yago-knowledge.org/resource/The_WB,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,The WB,The_WB,"The WB Television Network (for Warner Bros., o..."
3,http://yago-knowledge.org/resource/This_Must_B...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,This Must Be the Place (film),This_Must_Be_the_Place_(film),This Must Be the Place is a 2011 Italian-Frenc...
4,"http://yago-knowledge.org/resource/Monsters,_Inc.",http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,"Monsters, Inc.","Monsters,_Inc.","Monsters, Inc. (also known as Monsters, Incorp..."
...,...,...,...,...,...,...
12758,http://yago-knowledge.org/resource/Lincoln_Col...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,"Lincoln College, Oxford","Lincoln_College,_Oxford","Lincoln College (formally, The College of the ..."
12759,http://yago-knowledge.org/resource/104th_Unite...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,104th United States Congress,104th_United_States_Congress,The 104th United States Congress was a meeting...
12760,http://yago-knowledge.org/resource/London_Busi...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,London Business School,London_Business_School,London Business School (LBS) is a business sch...
12761,http://yago-knowledge.org/resource/Political_s...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,Political science,Political_science,Political science is the scientific study of p...


## Get ChatGPT Triples

In [ ]:
import openai

def chatGPTQuery(label,abstract):
    
    openai.api_key = "KEY"

    chatquery = ("""output just rdf like triples for the entity '%s' from the text:
                 %s""" %(label, abstract))
    
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": "You are a chatbot"},
            {"role": "user", "content": chatquery},
        ]
        )

    result = ''
    for choice in response.choices:
        result += choice.message.content

    return result

triples = []
for s,p,o,label,dbosource,abstract in data.values:
    if len(abstract) > 0:
        triples.append(chatGPTQuery(label,abstract))
    else:
        triples.append('')

data['Triples'] = triples
triples[0]

## Extract Triples from String

In [ ]:
# Need to see strings to see how to export

## Export triples

In [ ]:
# Export triples into .ttl file 
#  # add to ChatGPT version of graphDB